# Analysis & *.h5 saving of cells and backgrounds
### Johanna Rahm, Sebastian Malkusch
#### Research group Heilemann
#### Institute for Physical and Theoretical Chemistry, Goethe University Frankfurt a.M.
This notebook as to be reloaded before analysing a new group of data!

## Load *.trc files

Choose/insert a directory for the cell files to be searched in (e.g. F:\Marburg\single_colour_tracking\resting). All *.trc  files in the directory will be loaded for the analysis. It is possible to ignore files, by choosing mask words. If a file name containes any mask word, it will not be loaded. Comma separate multiple mask words (e.g. hmm, background, cell02).

In [1]:
from pySPT.widgets.widgetNotebooks import init_save_track_analysis
from pySPT.widgets import hdf5
from pySPT.widgets import widgetTrackAnalysis
from pySPT.widgets import widgetColumnSort
from pySPT.widgets import widgetDirectoryStructure
from pySPT.analysis import coverSlip
from pySPT.analysis import trackAnalysis
from pySPT.analysis import trcFormat
from tqdm import tqdm_notebook as tqdm
import warnings
widget_track_analysis = widgetTrackAnalysis.WidgetTrackAnalysis()
track_analysis = trackAnalysis.TrackAnalysis()
cover_slip = coverSlip.CoverSlip()
display(widget_track_analysis.software_button, widget_track_analysis.dir_box, widget_track_analysis.dir_button)
widget_track_analysis.dir_button.on_click(widget_track_analysis.open_dir)
widget_track_analysis.dir_box.observe(widget_track_analysis.change_dir_box)

RadioButtons(options=('ThunderSTORM', 'rapidSTORM', 'PALMTracer'), value='ThunderSTORM')

Text(value='directory to be searched in', description='directory', placeholder='Type something', style=Descrip…

Button(description='browse', style=ButtonStyle(), tooltip='browse for directory')

In [2]:
def test(event):
    widget_track_analysis.searchSubFolders(widget_track_analysis.dir_name)
    print(widget_track_analysis.file_names)
display(widget_track_analysis.test_button)
widget_track_analysis.test_button.on_click(test)

Button(description='test', style=ButtonStyle(), tooltip='testing')

Choose/insert the complete directory for the roi.log file which contains the cell sizes (e.g. F:\Marburg\single_colour_tracking\resting\roi.log). If no cell size was determined, the camera chip size will be set as the size.

In [3]:
display(widget_track_analysis.roi_box, widget_track_analysis.roi_button)
widget_track_analysis.roi_button.on_click(widget_track_analysis.open_roi)
widget_track_analysis.roi_box.observe(widget_track_analysis.change_roi_box)

Text(value='path of roi', description='roi', placeholder='Type something', style=DescriptionStyle(description_…

Button(description='browse', style=ButtonStyle(), tooltip='browse for roi')

## Camera details

Based on the pixel size, the localization will be transfered from pixel to micrometer.
Based on the amount of pixel, the camera area will be calculated (e.g. 256 * 256=65536 pixel). Camera integration time is needed for MSD calculations.

In [4]:
display(widget_track_analysis.camera_pixel_size_box, widget_track_analysis.camera_pixel_amount_box, widget_track_analysis.camera_integration_time_box)

Text(value='158', description='Pixel size [nm]', placeholder='Type something', style=DescriptionStyle(descript…

Text(value='65536', description='Amount of pixel on the camera', placeholder='Type something', style=Descripti…

Text(value='0.02', description='Camera integration time [s]', placeholder='Type something', style=DescriptionS…

## Parameters for Diffusion Type Analysis

The degree of freedom of the diffusion coefficient D are inserted in the calculation of D (1D = 2, 2D = 4, 3D = 6 dof). The minimal detectable diffusion coefficient is the threshold for the immobile type analysis.
Based on the formular min_track_length * camera_integration_time = n -> tau_threshold = 0.3*n, the tau threshold for diffusion type determination free & confined is set.

In [5]:
display(widget_track_analysis.points_D_fit_box, widget_track_analysis.dof_box, widget_track_analysis.D_min_box, widget_track_analysis.min_track_length_box)

Text(value='4', description='Number of points fitted for D', placeholder='Type something', style=DescriptionSt…

Text(value='4', description='degree of freedom of D', placeholder='Type something', style=DescriptionStyle(des…

Text(value='0.0065', description='Minimal detectable D  [μm²/s]', placeholder='Type something', style=Descript…

Text(value='20', description='Min track length', placeholder='Type something', style=DescriptionStyle(descript…

The diffusion type analysis can be based on the track id or seg id of the trc file. A track can contain multiple segmentations with different diffusion models that will be averaged. Therefore running the analysis on a segment level is recommended. For PALMTracer .trc files no seg id is available and the track id has to be chosen.

In [6]:
display(widget_track_analysis.trajectory_id_button)

RadioButtons(options=('seg id', 'track id'), value='seg id')

## Parameters for Hidden Markov Modeling

Check the box to enable hidden markov modeling. A .trc file for the ermine cluster will be saved in the pySPT/hmm folder. The hidden markov modeling is applied to the track id to observe state transformations. Choose a min track length to investigate the system (at least 2).

In [7]:
display(widget_track_analysis.hmm_check_box, widget_track_analysis.min_track_length_hmm_box)

Checkbox(value=True, description='Save .trc file')

Text(value='2', description='Min track length', placeholder='Type something', style=DescriptionStyle(descripti…

## Run Analysis

In [8]:
widget_dir_structure = widgetDirectoryStructure.WidgetDirStructure()#??

warnings.filterwarnings('ignore')
display(widget_track_analysis.run_button)
def run_analysis(event):
    widget_track_analysis.create_clear_output()
    widget_track_analysis.searchSubFolders(widget_track_analysis.dir_name)
    if widget_track_analysis.got_dir:
        cover_slip.software, cover_slip.min_track_length_type, cover_slip.min_track_length_hmm, cover_slip.dt, cover_slip.pixel_size, cover_slip.pixel_amount, cover_slip.dof, cover_slip.D_min, cover_slip.roi_file, cover_slip.cell_files, cover_slip.points_fit_D, cover_slip.seg_id = widget_track_analysis.software_button.value, widget_track_analysis.min_track_length_box.value, widget_track_analysis.min_track_length_hmm_box.value, widget_track_analysis.camera_integration_time_box.value, widget_track_analysis.camera_pixel_size_box.value, widget_track_analysis.camera_pixel_amount_box.value, widget_track_analysis.dof_box.value, widget_track_analysis.D_min_box.value, widget_track_analysis.roi_name, widget_track_analysis.file_names, widget_track_analysis.points_D_fit_box.value, widget_track_analysis.trajectory_id_button.value      
        for cell_idx in range(len(cover_slip.cell_files)):
            if widget_track_analysis.software_button.value != "PALMTracer":
                if widget_track_analysis.software_button.value == "ThunderSTORM":
                    widget_column_sort = widgetColumnSort.WidgetColumnSort(cover_slip.cell_files[cell_idx], widget_track_analysis.software_button.value, ['"track.id"' , '"x [nm]"', '"y [nm]"', '"frame"', '"intensity [photon]"', '"seg.id"'])
                elif widget_track_analysis.software_button.value == "rapidSTORM":
                    widget_column_sort = widgetColumnSort.WidgetColumnSort(cover_slip.cell_files[cell_idx], widget_track_analysis.software_button.value, ['"track.id"', '"Position-0-0"', '"Position-1-0"', '"Amplitude-0-0"', '"ImageNumber-0-0"', '"seg.id"'])
                widget_column_sort.check_header()
                if widget_column_sort.correct_header:
                    widget_column_sort.run_column_sort()
                    cover_slip.column_orders.append(widget_column_sort.column_order)
        cover_slip.create_cells()  
        track_analysis.cell_sizes = [cell.size for cell in cover_slip.cells]
        track_analysis.cell_trajectories = cover_slip.cell_trajectories
        track_analysis.run_statistics_no_filter()
    else:
        widget_track_analysis.warning_trc_file()
    widget_track_analysis.cells = cover_slip.cells
    for cell in cover_slip.cells:
        print("sigma dyn", cell.sigma_dyn)
widget_track_analysis.run_button.on_click(run_analysis)

suffix output ThunderSTORM tracked.csv
suffix output ThunderSTORM tracked.csv


          id  frame   x [nm]   y [nm]  sigma [nm]  intensity [photon]  \
0          1      1   2803.2  15391.7       179.7               233.2   
1          2      1   7193.6  22426.9       179.3               749.3   
2          3      1   7537.4  13718.4       154.4               556.4   
3          4      1   8409.5  16289.0        55.8               200.0   
4          5      1  11957.7  19358.4       132.5               200.0   
5          6      1  13707.5  17041.6       146.5              1312.5   
6          7      1  13962.5  21398.4       156.0               892.9   
7          8      1  15442.3  20833.4       142.8               935.3   
8          9      1  16349.3  17884.1       144.0               907.9   
9         10      1  17317.1   9764.8       139.2               606.7   
10        11      1  18132.4   5847.6       137.3               217.0   
11        12      1  22600.8   9051.7       146.2               200.0   
12        13      1  23868.9   6933.7       538.7  

[23010 rows x 29 columns]
Conversion successful.
<class 'list'>
<class 'list'>
[2, 1, 222.953164556962, 107.75316455696202, 0, 328.5, 2]


[[2, 1, 222.953164556962, 107.75316455696202, 0, 328.5, 2], [2, 2, 222.53164556962025, 107.6246835443038, 0, 490.4, 2], [2, 3, 221.7892405063291, 108.27784810126583, 0, 481.3, 2], [2, 4, 221.326582278481, 108.2626582278481, 0, 512.7, 2], [2, 5, 221.83607594936709, 108.4132911392405, 0, 765.5, 2], [2, 6, 221.49430379746835, 108.8854430379747, 0, 515.9, 2], [2, 7, 220.2, 109.3012658227848, 0, 530.9, 2], [2, 8, 220.42911392405065, 110.10822784810125, 0, 719.1, 2], [2, 9, 220.27721518987343, 109.87151898734177, 0, 613.8, 2], [2, 10, 220.38291139240508, 109.57974683544303, 0, 505.4, 2], [2, 11, 220.74746835443037, 109.8620253164557, 0, 495.2, 2], [2, 12, 220.82531645569622, 109.23164556962024, 0, 410.3, 2], [2, 13, 221.72658227848103, 108.71582278481011, 0, 457.7, 2], [2, 14, 221.33354430379745, 108.5120253164557, 0, 444.0, 2], [2, 15, 221.07215189873418, 108.07215189873419, 0, 619.3, 2], [2, 16, 221.18037974683546, 107.73164556962024, 0, 462.1, 2], [2, 17, 221.34936708860758, 106.934177215

cell tau 0.12


          id  frame   x [nm]   y [nm]  sigma [nm]  intensity [photon]  \
0          1      1   2517.3  22761.7       194.6               373.8   
1          2      1   4432.5  23871.2       159.6               535.2   
2          3      1   4557.4  25317.7       120.0               200.0   
3          4      1   4994.2  28922.7       195.5               783.0   
4          5      1   5411.6   1810.8       191.8               307.0   
5          6      1   7568.6    826.4       182.8               481.3   
6          7      1   7895.7   1241.8       285.1               682.5   
7          8      1   7115.7    671.6       666.5              2500.0   
8          9      1   8214.2  17897.8       158.0               200.0   
9         10      1   8323.4  25848.4       159.0              1074.9   
10        11      1   8585.6   7898.8       244.0               359.2   
11        12      1   9787.7  30386.6       165.7               633.8   
12        13      1  10247.5  12213.6       158.9  

[21974 rows x 29 columns]
Conversion successful.
<class 'list'>
<class 'list'>
[2, 1, 94.02848101265823, 156.2120253164557, 0, 780.5, 2]


[[2, 1, 94.02848101265823, 156.2120253164557, 0, 780.5, 2], [2, 2, 93.90253164556962, 156.01962025316456, 0, 624.0, 2], [2, 3, 93.17911392405063, 155.95, 0, 729.9, 2], [2, 4, 93.51139240506329, 155.70189873417723, 0, 739.1, 2], [2, 5, 93.33607594936709, 155.68291139240506, 0, 710.0, 2], [2, 6, 93.80253164556962, 155.57025316455696, 0, 670.1, 2], [2, 7, 93.94113924050633, 155.19430379746836, 0, 637.6, 2], [2, 8, 93.92911392405063, 155.32088607594937, 0, 421.5, 2], [2, 9, 94.03354430379747, 155.22215189873415, 0, 571.6, 2], [2, 10, 94.23797468354431, 154.93354430379748, 0, 489.6, 2], [2, 11, 94.15189873417721, 154.94367088607595, 0, 444.1, 2], [2, 12, 94.0493670886076, 154.72025316455696, 0, 489.2, 2], [2, 13, 93.726582278481, 154.7240506329114, 0, 438.3, 2], [2, 14, 93.52278481012658, 154.48037974683544, 0, 701.8, 2], [2, 15, 93.42025316455695, 155.1481012658228, 0, 513.0, 2], [2, 16, 93.16075949367088, 155.36455696202532, 0, 651.1, 2], [2, 17, 93.5386075949367, 155.22151898734177, 0, 4

cell tau 0.12



Analysis took 12.365094184875488 s
23.9 % are immobile
14.2 % are confined
60.3 % are free
Total trajectories: 401
sigma dyn 0.029686476344628836
sigma dyn 0.03371066619537645


## Plot diffusion histogram

Plot the normalized frequency in % against the diffusion coefficients (semi log scale). Diffusion coefficients <= 0 are neglected, because the logarithm is not defined in this interval.

In [9]:
display(widget_track_analysis.bin_size_box)

Text(value='0.1', description='bin size', placeholder='size for log10(D) histogram', style=DescriptionStyle(de…

In [10]:
def plot_diffusions(event):
    widget_track_analysis.create_clear_output()
    display(widget_track_analysis.plot_diff_button)
    track_analysis.run_plot_diffusion_histogram(widget_track_analysis.bin_size_box.value)
display(widget_track_analysis.plot_diff_button)
widget_track_analysis.plot_diff_button.on_click(plot_diffusions)

Button(description='plot', style=ButtonStyle(), tooltip='plot diffusion histogram')

## Choose trajectory to plot

Choose a cell. If one wants to plot multiple times, choose a new cell and/or trajectory reload the plotting cell by clicking in the cell and pressing strg+enter.

In [11]:
def dropdown(event):
    widget_track_analysis.cell_options = widget_track_analysis.get_cell_names(cover_slip.cells)
widget_track_analysis.run_button.on_click(dropdown)
display(widget_track_analysis.drop_down_cells)

Dropdown(description='Number:', options=(), value=None)

Choose a trajectory.

In [12]:
def dropdown(event):      
    for cell in cover_slip.cells:
        if cell.name == widget_track_analysis.drop_down_cells.value:
            target_cell = cover_slip.cells.index(cell)
            trajectory_indexes = [i+1 for i, _ in enumerate(cover_slip.cell_trajectories[target_cell])]
            widget_track_analysis.trajectory_options = trajectory_indexes
            widget_track_analysis.drop_down_trajectories.options = widget_track_analysis.trajectory_options
widget_track_analysis.drop_down_cells.observe(dropdown)
display(widget_track_analysis.drop_down_trajectories)

Dropdown(description='Number:', options=(), value=None)

In [13]:
def plot_particle(event):
    widget_track_analysis.create_clear_output()
    display(widget_track_analysis.plot_button)
    cover_slip.plot_trajectory(widget_track_analysis.drop_down_cells.value, widget_track_analysis.drop_down_trajectories.value)
display(widget_track_analysis.plot_button)
widget_track_analysis.plot_button.on_click(plot_particle)

Button(description='plot', style=ButtonStyle(), tooltip='plot chosen trajectory')

## Save

For each cell a .h5 file will be created, containing all information of the trajectories (unfiltered).

In [14]:
def save_analysis(event):    
    for cell_index in tqdm(range(0, len(cover_slip.cells))):
        init_save_track_analysis(cover_slip, cell_index, track_analysis, widget_track_analysis.points_D_fit_box.value)
display(widget_track_analysis.save_button)
widget_track_analysis.save_button.on_click(save_analysis)

Button(description='save', style=ButtonStyle(), tooltip='save entire analysis')